# Week 2 Assignment: ARIMA vs LSTM for Stock Price Prediction

This notebook implements **both ARIMA and LSTM models** for stock price prediction and provides a **comparative analysis** between statistical and deep learning approaches.

The workflow follows the assignment requirements step by step with explanations, visualizations, and evaluation metrics.


## 0) Setup and Imports

In [ ]:

# Uncomment if required
# !pip install yfinance tensorflow pmdarima scikit-learn

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

plt.rcParams['figure.figsize'] = (12,5)


## 1) Data Collection

In [ ]:

import yfinance as yf

TICKER = 'MSFT'   # Change if required
START_DATE = '2019-01-01'
END_DATE = '2024-12-31'

data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)
df = data[['Adj Close', 'Volume']].copy()
df.dropna(inplace=True)

df.head()


In [ ]:

plt.plot(df['Adj Close'])
plt.title(f'{TICKER} Adjusted Close Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()


## 2) Data Preprocessing

In [ ]:

# Check missing values
print(df.isna().sum())

# Stationarity check (ADF test)
adf_result = adfuller(df['Adj Close'])
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])


### Train-Test Split (80%-20%)

In [ ]:

n = len(df)
train_size = int(n * 0.8)

train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

print(len(train_df), len(test_df))


## 3) ARIMA Model

In [ ]:

plot_acf(train_df['Adj Close'], lags=40)
plot_pacf(train_df['Adj Close'], lags=40)
plt.show()


In [ ]:

# Fit ARIMA model
arima_model = ARIMA(train_df['Adj Close'], order=(1,1,1))
arima_fit = arima_model.fit()

print(arima_fit.summary())


In [ ]:

# Forecast
arima_forecast = arima_fit.forecast(steps=len(test_df))

plt.plot(test_df.index, test_df['Adj Close'], label='Actual')
plt.plot(test_df.index, arima_forecast, label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Forecast vs Actual')
plt.show()


In [ ]:

# ARIMA Evaluation
arima_mae = mean_absolute_error(test_df['Adj Close'], arima_forecast)
arima_rmse = mean_squared_error(test_df['Adj Close'], arima_forecast, squared=False)

print('ARIMA MAE:', arima_mae)
print('ARIMA RMSE:', arima_rmse)


## 4) LSTM Model

In [ ]:

# Normalize data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_prices = scaler.fit_transform(df[['Adj Close']])


### Sliding Window Creation

In [ ]:

def create_sequences(data, window=60):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

WINDOW_SIZE = 60

X, y = create_sequences(scaled_prices, WINDOW_SIZE)

X_train, X_test = X[:train_size-WINDOW_SIZE], X[train_size-WINDOW_SIZE:]
y_train, y_test = y[:train_size-WINDOW_SIZE], y[train_size-WINDOW_SIZE:]

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


### Build and Train LSTM

In [ ]:

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


### LSTM Predictions

In [ ]:

lstm_preds = model.predict(X_test)
lstm_preds = scaler.inverse_transform(lstm_preds)

actual_prices = df['Adj Close'].iloc[train_size:].values.reshape(-1,1)


In [ ]:

plt.plot(actual_prices, label='Actual')
plt.plot(lstm_preds, label='LSTM Predictions')
plt.legend()
plt.title('LSTM Forecast vs Actual')
plt.show()


### LSTM Evaluation

In [ ]:

lstm_mae = mean_absolute_error(actual_prices, lstm_preds)
lstm_rmse = mean_squared_error(actual_prices, lstm_preds, squared=False)

print('LSTM MAE:', lstm_mae)
print('LSTM RMSE:', lstm_rmse)


## 5) ARIMA vs LSTM Comparison

In [ ]:

comparison = pd.DataFrame({
    'Model': ['ARIMA', 'LSTM'],
    'MAE': [arima_mae, lstm_mae],
    'RMSE': [arima_rmse, lstm_rmse]
})

comparison


## 6) LSTM Learning Curves

In [ ]:

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('LSTM Learning Curve')
plt.show()


## 7) Conclusions

**ARIMA**
- Works well for linear, stationary time series
- Low computational cost
- Limited ability to capture non-linear patterns

**LSTM**
- Captures complex non-linear dependencies
- Handles long-term memory effectively
- Requires more data and computation

**Future Improvements**
- Bidirectional LSTM
- Technical indicators as features
- Hyperparameter tuning
